In [1]:
import snap
import time, gensim, pickle, json, networkx as nx, numpy as np
from networkx.readwrite import json_graph

In [2]:
with open('../REST/static/networks/twitter_users_graph2.json', 'r') as f:
    data = json.load(f)
    nxg = json_graph.node_link_graph(data, directed=True)
nxg = nxg.to_directed()

In [3]:
g = snap.PUNGraph.New()

In [4]:
for n in nxg.nodes():
    g.AddNode(n)

In [5]:
for f,t in nxg.edges():
    g.AddEdge(f, t)

In [6]:
CmtyV = snap.TCnComV()

In [7]:
start = time.time()
modularity = snap.CommunityGirvanNewman(g, CmtyV)
print(time.time()-start)

140.32566309


In [8]:
nodes_communities = {} # {node: [community]}
for i, Cmty in enumerate(CmtyV):
    print ("Community: ")
    for NI in Cmty:
        nodes_communities.setdefault(NI, [])
        nodes_communities[NI].append(i)
        print (NI)
print ("The modularity of the network is %f" % modularity)

Community: 
9434942
14668733
74959905
103012327
103123766
133943907
137323878
147180161
157616590
179238549
194196967
202625973
218862394
221355069
236914275
239980933
249747208
253229470
253777838
257503461
259404430
263167320
269014960
271718939
278750314
287584045
298713229
299384045
301378912
304139482
308041033
329880363
330546053
336939514
367968322
369836872
398652596
412005944
425933822
445806392
495928885
521494559
529401410
574630629
599159655
747510565
756224243
766219389
777958030
796448176
800981689
818782100
819172968
824302600
894491384
907154399
935460342
947934061
962844904
971044607
Community: 
21803514
66713738
81448875
106785364
108271772
110320190
113712545
144907443
151050084
157049672
159630069
175426030
193879875
219157616
220145170
228277695
235114569
238165726
239746679
243882007
275966437
276882364
281127185
287730356
295464374
313467951
314320500
318339274
321885193
333501343
336584201
337659666
362755027
376697378
405717491
448053871
452106609
489239437
501

In [9]:
for node in nxg.nodes():
    nxg.nodes[node]["community"] = " ".join([str(com) for com in nodes_communities[node]])

In [10]:
for ix,deg in nxg.degree(nxg.nodes()):
    nxg.node[ix]['degree'] = deg

for ix,in_deg in nxg.in_degree(nxg.nodes()):
    nxg.node[ix]['in_degree'] = in_deg
    
for ix,out_deg in nxg.out_degree(nxg.nodes()):
    nxg.node[ix]['out_degree'] = out_deg

pagerank = nx.pagerank(nxg)

for ix,v in pagerank.items():
    nxg.node[ix]["pagerank"] = v

In [11]:
def networkx_to_snappy(nxg, directed=False):
    if directed:
        g = snap.PNGraph.New()
    else:
        g = snap.PUNGraph.New()
        
    for n in nxg.nodes():
        g.AddNode(n)
    for f,t in nxg.edges():
        g.AddEdge(f, t)
        
    return g

In [12]:
data = nx.node_link_data(nxg)
data["modularity"] = modularity
data["number_of_communities"] = len(CmtyV)

snappy_directed = networkx_to_snappy(nxg, True)
snappy_undirected = networkx_to_snappy(nxg, True)

diam_directed = snap.GetBfsFullDiam(snappy_directed, 100, True)
diam_undirected = snap.GetBfsFullDiam(snappy_undirected, 100, False)

data["diameter_directed"]=diam_directed
data["diameter_undirected"]=diam_undirected
with open('../visualization/twitter_users_graph2.json', 'w') as f:
    json.dump(data, f, indent=4)
    
with open('../REST/static/networks/twitter_users_graph2.json', 'w') as f:
    json.dump(data, f, indent=4)

In [13]:
json_graph.node_link_data(json_graph.node_link_graph(data, directed=True)).keys()

['directed', 'graph', 'nodes', 'links', 'multigraph']